<a href="https://colab.research.google.com/github/ganjiron/ganpython/blob/master/CNNwithBayesianOptimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.datasets import cifar10
from keras.utils import to_categorical
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split

# 데이터 로드 및 전처리
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)
x_train, x_val, x_test = x_train / 255.0, x_val / 255.0, x_test / 255.0
y_train, y_val, y_test = to_categorical(y_train), to_categorical(y_val), to_categorical(y_test)

In [17]:
pip install bayesian-optimization

In [19]:
def build_model(dropout_rate, learning_rate):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dropout(dropout_rate))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [20]:
def fit_with(dropout_rate, learning_rate):
    model = build_model(dropout_rate, learning_rate)
    model.fit(x_train, y_train, epochs=5, validation_data=(x_val, y_val), verbose=0)
    _, accuracy = model.evaluate(x_val, y_val, verbose=0)
    return accuracy

In [21]:
# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(
    f=fit_with,
    pbounds={
        'dropout_rate': (0.1, 0.5),  # 드롭아웃 비율 범위
        'learning_rate': (1e-4, 1e-2)  # 학습률 범위
    },
    random_state=1234
)

# 최적화 실행
optimizer.maximize(
    init_points=2,  # 초기 랜덤 포인트
    n_iter=5,       # 최적화 단계 수
)

# 최적의 하이퍼파라미터 출력
print(optimizer.max)

|   iter    |  target   | dropou... | learni... |
-------------------------------------------------
| 1         | 0.6732    | 0.1766    | 0.006259  |
| 2         | 0.6424    | 0.2751    | 0.007875  |
| 3         | 0.6635    | 0.285     | 0.003652  |
| 4         | 0.6428    | 0.254     | 0.005433  |
| 5         | 0.6542    | 0.3978    | 0.003587  |
| 6         | 0.6747    | 0.174     | 0.006417  |
| 7         | 0.6652    | 0.1581    | 0.0001    |
{'target': 0.6747000217437744, 'params': {'dropout_rate': 0.17400739760630013, 'learning_rate': 0.00641677200574913}}


Convolutional Neural Network (CNN)에서 Convolution Layer(컨볼루션 레이어)를 쌓을 때, 주요 구성 요소와 그들의 의미 및 변화 방법을 아래와 같이 설명할 수 있습니다.

1. Filters (필터)

	•	의미: 필터는 입력 데이터에서 특징을 추출하는 핵심 요소입니다. 각 필터는 입력 데이터의 다른 특징을 학습합니다.
	•	변화 방법: 초기 레이어에는 적은 수의 필터를 사용하고, 네트워크가 깊어질수록 필터의 수를 늘려가는 것이 일반적입니다. 예를 들어, 첫 번째 Conv 레이어에는 32개의 필터를 사용하고, 다음 레이어에는 64개, 그 다음에는 128개 등으로 늘립니다.

2. Kernel Size (커널 크기)

	•	의미: 커널 크기는 필터의 차원을 결정합니다. 일반적으로 3x3 또는 5x5 크기가 자주 사용됩니다.
	•	변화 방법: 작은 커널 크기는 더 세밀한 특징을, 큰 커널 크기는 넓은 영역의 특징을 추출합니다. 대부분의 경우 3x3 또는 5x5 크기를 사용합니다.

3. Kernel Regularizer (커널 정규화)

	•	의미: 정규화는 과적합을 방지하는 데 도움이 됩니다. L1, L2 정규화 등이 일반적입니다.
	•	변화 방법: 정규화를 추가하여 모델이 복잡해지는 것을 방지할 수 있습니다. 예를 들어, kernel_regularizer=tf.keras.regularizers.l2(0.01)과 같이 사용합니다.

4. Pooling (풀링)

	•	의미: 풀링은 특징 맵의 크기를 줄이면서 중요한 정보를 유지합니다. Max Pooling과 Average Pooling이 일반적입니다.
	•	변화 방법: 일반적으로 컨볼루션 레이어 다음에 풀링 레이어를 추가합니다. 예를들어, MaxPooling2D(pool_size=(2, 2))는 2x2 크기의 윈도우로 최대값을 추출하는 맥스 풀링을 적용합니다.

5. Activation (활성화 함수)

	•	의미: 활성화 함수는 신경망에 비선형성을 도입합니다. 이는 모델이 복잡한 패턴을 학습할 수 있게 해줍니다.
	•	변화 방법: ReLU는 가장 일반적으로 사용되는 활성화 함수입니다. 깊은 레이어에서는 LeakyReLU나 ELU 같은 변형된 형태를 사용하기도 합니다.

6. Dropout Rate (드롭아웃 비율)

	•	의미: 드롭아웃은 훈련 과정에서 무작위로 뉴런의 일부를 비활성화하여 과적합을 방지합니다.
	•	변화 방법: 드롭아웃 비율은 일반적으로 0.2에서 0.5 사이입니다. 과적합이 관찰되면 드롭아웃 비율을 증가시킬 수 있습니다.

일반적인 CNN 구조에서 이러한 요소들의 변화

	•	초기 레이어: 적은 수의 필터와 작은 커널 크기로 시작하여 세밀한 특징을 추출합니다. 드롭아웃은 일반적으로 사용하지 않거나 낮은 비율을 사용합니다.
	•	중간 레이어: 필터의 수를 증가시키고, 필요에 따라 커널 크기를 조정합니다. 정규화 또는 드롭아웃을 추가하여 과적합을 방지합니다.
	•	마지막 레이어: 클래스 분류를 위해 Flatten 후 Dense 레이어를 추가합니다. 이 때 Softmax 활성화 함수(다중 분류) 또는 Sigmoid 활성화 함수(이진 분류)를 사용합니다.

이러한 요소들을 적절히 조정하면 다양한 데이터와 문제에 맞는 효과적인 CNN 모델을 구축할 수 있습니다.


1000개의 사진을 이용하여 CNN(Convolutional Neural Network) 모델을 구축할 때 적절한 레이어의 수는 여러 요인에 따라 달라질 수 있습니다. 아래 요인들을 고려하여 결정하는 것이 중요합니다:

1. 이미지의 복잡성

	•	간단한 이미지: 단순한 이미지(예: 숫자 인식)의 경우, 몇 개의 컨볼루션 레이어만으로도 충분할 수 있습니다.
	•	복잡한 이미지: 복잡한 이미지(예: 자연 경관)나 다양한 객체를 포함하는 경우, 더 많은 레이어가 필요할 수 있습니다.

2. 데이터 양

	•	1000개의 이미지는 상대적으로 적은 양입니다. 너무 깊은 네트워크는 과적합(overfitting)을 일으킬 수 있습니다. 따라서 단순한 구조를 시작점으로 삼고 필요에 따라 점차 복잡성을 늘리는 것이 좋습니다.

3. 과적합 방지 전략

	•	데이터 증강(augmentation), 드롭아웃(dropout), 정규화(regularization)와 같은 과적합 방지 기법을 적용하여, 더 깊은 모델을 사용할 수 있습니다.

4. 실험과 조정

	•	레이어의 수를 조정하며 실험을 여러 번 해보고, 검증 데이터셋에서의 성능을 확인합니다. 처음에는 간단한 모델로 시작하여 점차 복잡도를 늘려가며 최적의 구조를 찾습니다.

일반적인 권장 사항

	•	초기 모델: 2-3개의 컨볼루션 레이어를 포함하는 간단한 모델로 시작합니다. 각 컨볼루션 레이어 뒤에는 활성화 함수(예: ReLU)와 풀링 레이어(MaxPooling)를 추가할 수 있습니다.

	•	중간 레이어: 필요에 따라 추가 컨볼루션 레이어를 추가하며, 각 레이어의 필터 수를 점차 늘려갑니다. 예를 들어, 첫 번째 레이어에서 32개의 필터를 사용한 후, 다음 레이어에서는 64개, 그 다음 레이어에서는 128개의 필터를 사용할 수 있습니다.
	•	드롭아웃과 정규화: 과적합을 방지하기 위해 드롭아웃 레이어를 추가하거나, 컨볼루션 레이어에 L1 또는 L2 정규화를 적용할 수 있습니다.
	•	완전 연결 레이어(Fully Connected Layer): 마지막에 하나 또는 두 개의 완전 연결 레이어(Dense Layer)를 추가하여 분류 작업을 수행합니다.

예시

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


이 예시에서는 두 개의 컨볼루션 레이어를 사용하고 있으며, 각 레이어 뒤에 맥스 풀링 레이어가 있습니다. 데이터 양이 비교적 적기 때문에, 모델이 너무 복잡하지 않도록 유의하는 것이 중요합니다.

이미 훈련된 모델에 새로운 데이터를 지속적으로 추가하면서 학습을 계속하는 방법은 “점진적 학습” 또는 “지속적 학습(continuous learning)“이라고 불리며, 특정 상황에서 유용할 수 있습니다. 이 방식은 특히 데이터가 시간에 따라 점진적으로 수집되는 경우나, 새로운 종류의 데이터를 모델에 통합해야 할 때 적합합니다.

장점

	1.	데이터 활용 최대화: 새로운 데이터가 수집될 때마다 이를 학습에 활용하여 모델의 성능을 개선할 수 있습니다.
	2.	시간과 자원 절약: 처음부터 모델을 다시 훈련시키지 않고, 기존에 학습된 가중치를 기반으로 추가 학습을 진행할 수 있어 시간과 자원을 절약할 수 있습니다.
	3.	새로운 특징 학습: 새로운 데이터에 포함된 새로운 특징이나 패턴을 모델이 학습할 수 있습니다.

단점 및 고려사항

	1.	과적합의 위험: 새로운 데이터에만 지나치게 특화되어 기존에 학습한 데이터에 대한 성능이 저하될 수 있습니다. 이를 “카타스트로픽 포기팅(catastrophic forgetting)“이라고 합니다.
	2.	데이터 분포의 변화: 새로운 데이터가 기존 데이터와 분포가 다를 경우, 모델이 이전 데이터에서 학습한 정보를 “잊어버릴” 수 있습니다.
	3.	지속적인 모니터링 필요: 모델이 계속해서 좋은 성능을 유지하고 있는지 정기적으로 확인하고, 필요에 따라 하이퍼파라미터를 조정해야 합니다.

접근 방법

	•	데이터 증강과 병합: 새로운 데이터와 기존 데이터를 적절히 혼합하여 사용하면, 과적합과 카타스트로픽 포기팅을 어느 정도 방지할 수 있습니다.
	•	학습률 조정: 추가 학습 시학습률을 낮추어 기존에 학습한 가중치를 과도하게 변경하지 않도록 조심스럽게 접근하는 것이 좋습니다. 너무 높은 학습률은 기존에 학습한 특징을 빠르게 잊게 만들 수 있습니다.

	•	적절한 평가: 지속적인 학습 과정에서는 새로운 데이터 뿐만 아니라 기존 데이터에 대해서도 모델의 성능을 주기적으로 평가해야 합니다. 이를 통해 모델이 전체 데이터셋에 대해 균형 잡힌 성능을 유지하고 있는지 확인할 수 있습니다.
	•	분할 학습: 새로운 데이터가 매우 다른 특징을 가지고 있다면, 별도의 모델을 훈련시키고 이를 기존 모델과 앙상블하는 방법도 고려할 수 있습니다.

결론

지속적으로 데이터가 추가될 때, 기존 모델에 새로운 데이터를 반복적으로 학습시키는 것은 유용할 수 있지만, 과적합, 카타스트로픽 포기팅, 데이터 분포의 변화 등에 주의해야 합니다. 적절한 학습률 설정, 데이터 증강, 주기적인 평가를 통해 이러한 문제를 최소화할 수 있습니다.